In [1]:
import numpy as np
import json
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import imageio
import mapclassify
import cv2
import os

# geopandas - choropleth maps

In [2]:
broj_pozitivnih = []
number = [1, 2, 3, 4, 5, 6, 7, 8, 9]
fix_border = ['1,', '2,', '3,', '4,' ,'5,' ,'6,', '7,', '8,', '9,']
PATH = 'results/'

In [3]:
def custom_date(date):
    return date[8:10]+'.'+date[5:7]+'.'+date[0:4]+'.'

In [4]:
def rename_geodata(croatia_data):
    croatia_data.rename(columns = {'NAME_1' : 'Zupanija'}, inplace=True)
    
    croatia_data.replace('Bjelovarska-Bilogorska', 'Bjelovarsko-bilogorska', inplace=True)
    croatia_data.replace('Brodsko-Posavska', 'Brodsko-posavska', inplace=True)
    croatia_data.replace('Dubrovacko-Neretvanska', 'Dubrovačko-neretvanska', inplace=True)
    croatia_data.replace('Karlovacka', 'Karlovačka', inplace=True)
    croatia_data.replace('Koprivničko-Križevačka', 'Koprivničko-križevačka', inplace=True)
    croatia_data.replace('Krapinsko-Zagorska', 'Krapinsko-zagorska', inplace=True)
    croatia_data.replace('Licko-Senjska', 'Ličko-senjska', inplace=True)
    croatia_data.replace('Medimurska', 'Međimurska', inplace=True)
    croatia_data.replace('Osjecko-Baranjska', 'Osječko-baranjska', inplace=True)
    croatia_data.replace('Požeško-Slavonska', 'Požeško-slavonska', inplace=True)
    croatia_data.replace('Primorsko-Goranska', 'Primorsko-goranska', inplace=True)
    croatia_data.replace('Sisacko-Moslavacka', 'Sisačko-moslavačka', inplace=True)
    croatia_data.replace('Splitsko-Dalmatinska', 'Splitsko-dalmatinska', inplace=True)
    croatia_data.replace('Viroviticko-Podravska', 'Virovitičko-podravska', inplace=True)
    croatia_data.replace('Vukovarsko-Srijemska', 'Vukovarsko-srijemska', inplace=True)
    croatia_data.replace('Šibensko-Kninska', 'Šibensko-kninska', inplace=True)
    croatia_data.replace('Zagrebacka', 'Zagrebačka', inplace=True)

In [5]:
def convert_to_df(df):
    df = pd.DataFrame(df, columns = ['Zupanija', 'broj_zarazenih', 'broj_umrlih', 'broj_aktivni', 'pozitivni'])
    return df

In [6]:
def prepare_covidPositive_data(df):
    df = df.sort_values(by=['Datum'], ignore_index = True) # ignore_index = resulting axis will be labeled 0, 1, ..., n-1
    for i in range(len(df.index)): # po svim zapisima --> len(df.index)
        #print(len(df.index))           
        for j in range(21): # po zupanijama
            if (i == 0): # iznimka prvi dan
                df.loc[i][3][j]['pozitivni'] = df.loc[i][3][j]['broj_zarazenih']
            elif (df.loc[i][3][j]['broj_zarazenih'] > df.loc[i-1][3][j]['broj_zarazenih']):
                df.loc[i][3][j]['pozitivni'] = df.loc[i][3][j]['broj_zarazenih'] - df.loc[i-1][3][j]['broj_zarazenih']
            else:
                df.loc[i][3][j]['pozitivni'] = 0
            broj_pozitivnih.append(df.loc[i][3][j]['pozitivni'])
            
            # pozitivni = podatak koji sadrži broj novozarazenih tog dana
    return df

In [7]:
def fix_legend_border(legend_border):
    for i in legend_border:
        i.set_text(i.get_text().replace(' ', ''))
        if i.get_text()[:2] in fix_border:
            tmp = i.get_text()[:2]
            i.set_text(i.get_text().replace(tmp, '0,'))
        if (i.get_text()[:5] == '-inf,'):
            tmp = i.get_text()[:5]
            i.set_text(i.get_text().replace(tmp, '0,'))
        i.set_text(i.get_text().replace(',', ' - '))

In [8]:
def visualize_covidPositive_data(df, date):
    merged_data = croatia_data.merge(df, on = 'Zupanija')
    #merged_data.to_file(r'shapefiles/merged_data/combined.shp')
    font_title = {'fontname':'sans-serif', 'color':'black', 'size':12, 'weight':'light'}
    fig, ax = plt.subplots(1, 1)
    ax.axis('off')
    merged_data.plot(cmap = 'YlOrBr', edgecolor = 'black', linewidth = 0.18, column = 'pozitivni', ax=ax, 
                     legend = True, scheme = 'user_defined', vmin = 0,
                     classification_kwds = {'bins':[10, 25, 50, 100, 250, 500, 1000, 1250, 1500]},
                     legend_kwds = {'loc': 'center left', 'bbox_to_anchor':(1,0.5), 'fmt': '{:.0f}'})
    
    fix_legend_border(ax.get_legend().texts)
        
    fig.colorbar  
    plt.title(custom_date(date), fontdict = font_title, backgroundcolor = 'floralwhite')
    plt.suptitle('Dnevni broj COVID-19 pozitivnih slučajeva' + '\n po županijama u RH', fontsize=8, 
                 weight = 'light', backgroundcolor = 'floralwhite', x = 0.25, y = 0.2)
    plt.savefig(PATH + date, dpi=70)
    plt.close();

In [9]:
croatia_data = gpd.read_file('shapefiles/croatia/HRV_adm1.shp')
rename_geodata(croatia_data)

f = pd.read_json('data/data_07_10_2021.json')
df = prepare_covidPositive_data(pd.DataFrame(f))

for i in range(len(df.index)):
    for j in df.loc[i]:
        per_day_data = convert_to_df(df.loc[i][3])
        visualize_covidPositive_data(per_day_data, df.loc[i][2][:-5]) ## yyyy-mm-dd

/home/lena/.local/lib/python3.8/site-packages/mapclassify/classifiers.py:891: RuntimeWarning: invalid value encountered in double_scalars
  gadf = 1 - self.adcm / adam
/home/lena/.local/lib/python3.8/site-packages/mapclassify/classifiers.py:891: RuntimeWarning: invalid value encountered in double_scalars
  gadf = 1 - self.adcm / adam
/home/lena/.local/lib/python3.8/site-packages/mapclassify/classifiers.py:891: RuntimeWarning: invalid value encountered in double_scalars
  gadf = 1 - self.adcm / adam
/home/lena/.local/lib/python3.8/site-packages/mapclassify/classifiers.py:891: RuntimeWarning: invalid value encountered in double_scalars
  gadf = 1 - self.adcm / adam
/home/lena/.local/lib/python3.8/site-packages/mapclassify/classifiers.py:891: RuntimeWarning: invalid value encountered in double_scalars
  gadf = 1 - self.adcm / adam
/home/lena/.local/lib/python3.8/site-packages/mapclassify/classifiers.py:891: RuntimeWarning: invalid value encountered in double_scalars
  gadf = 1 - self.adcm

/home/lena/.local/lib/python3.8/site-packages/mapclassify/classifiers.py:891: RuntimeWarning: invalid value encountered in double_scalars
  gadf = 1 - self.adcm / adam
/home/lena/.local/lib/python3.8/site-packages/mapclassify/classifiers.py:891: RuntimeWarning: invalid value encountered in double_scalars
  gadf = 1 - self.adcm / adam
/home/lena/.local/lib/python3.8/site-packages/mapclassify/classifiers.py:891: RuntimeWarning: invalid value encountered in double_scalars
  gadf = 1 - self.adcm / adam
/home/lena/.local/lib/python3.8/site-packages/mapclassify/classifiers.py:891: RuntimeWarning: invalid value encountered in double_scalars
  gadf = 1 - self.adcm / adam
/home/lena/.local/lib/python3.8/site-packages/mapclassify/classifiers.py:891: RuntimeWarning: invalid value encountered in double_scalars
  gadf = 1 - self.adcm / adam
/home/lena/.local/lib/python3.8/site-packages/mapclassify/classifiers.py:891: RuntimeWarning: invalid value encountered in double_scalars
  gadf = 1 - self.adcm

In [10]:
#print(broj_pozitivnih)
print(max(broj_pozitivnih))
print(min(broj_pozitivnih))
print(np.isnan(broj_pozitivnih))

1415
0
[False False False ... False False False]


In [11]:
pathdir = './' + PATH
images = []
for filename in sorted(os.listdir(pathdir)):
    if filename.endswith('.png'):
        filepath = os.path.join(pathdir, filename)
        images.append(imageio.imread(filepath))
        images.append(imageio.imread(filepath))
imageio.mimsave(pathdir + 'movie.gif', images)